In [1]:
import pandas as pd
import geopandas as gpd
import folium

# 1. Verileri Okuma
participant_path = r"C:\Users\samet\Downloads\BIOCOM_24_Kayit_Formu_UTF8.xlsx"
uni_path = r"C:\Users\samet\Downloads\Duzenlenmis_Universite_Listesi.xlsx"
geojson_path = r"C:\Users\samet\Downloads\tr-cities.geojson"

participant_data = pd.read_excel(participant_path)
uni_data = pd.read_excel(uni_path)
city_coords = gpd.read_file(geojson_path)

# Sütun isimlerini kontrol edelim:
print("Katılımcı verisinin sütunları:", participant_data.columns.tolist())
print("Üniversite verisinin sütunları:", uni_data.columns.tolist())
print("GeoJSON verisinin sütunları:", city_coords.columns.tolist())

# 2. Üniversite İsimlerini Normalleştirme ve Ayrıştırma

# Kısaltma eşleştirme sözlüğü (KTÜ gibi kısaltmaları da ekliyoruz)
abbr_mapping = {
    "ODTÜ": "Orta Doğu Teknik Üniversitesi",
    "Boğaziçi": "Boğaziçi Üniversitesi",
    "ITU": "İstanbul Teknik Üniversitesi",
    "KTÜ": "Karadeniz Teknik Üniversitesi",
    "İYTE":"İzmir Yüksek Teknoloji Enstitüsü"
    # Diğer kısaltmaları ekleyin...
}

def normalize_university(x):
    """
    - Eğer değer NaN ise boş string döndür.
    - Eğer "lise" kelimesi varsa, bu katılımcıyı "Lise" olarak işaretle.
    - Aksi halde, önce sözlükten tam isim arar; 
      eğer sonuçta 'üniversite' ya da 'university' ifadesi geçiyorsa normal üniversite,
      yoksa "Yabancı" olarak işaretle.
    """
    if pd.isna(x):
        return ""
    if not isinstance(x, str):
        x = str(x)
    orig = x.strip()
    # Yüksek liseli kontrolü (küçük harfe çevirip "lise" arıyoruz)
    if "lise" in orig.lower():
        return "Lise"
    # Sözlük üzerinden normalizasyon
    normalized = abbr_mapping.get(orig, orig)
    # Eğer sonuçta 'üniversite' (veya 'university') kelimesi varsa normal kabul et, yoksa yabancı
    if ("üniversite" in normalized.lower()) or ("university" in normalized.lower()):
        return normalized
    else:
        return "Yabancı"

# Yeni sütun: Normalized_University
participant_data["Normalized_University"] = participant_data["Okulunuz/Kurumunuz"].apply(normalize_university)

# 3. Katılımcıları Kategorilere Ayıralım
normal_participants = participant_data[~participant_data["Normalized_University"].isin(["Lise", "Yabancı"])]
lise_participants   = participant_data[participant_data["Normalized_University"] == "Lise"]
yabanci_participants = participant_data[participant_data["Normalized_University"] == "Yabancı"]

print("Normal Üniversite Katılımcı Sayısı:", len(normal_participants))
print("Lise Katılımcı Sayısı:", len(lise_participants))
print("Yabancı Katılımcı Sayısı:", len(yabanci_participants))
print("Toplam Katılımcı Sayısı:", len(participant_data))

# 4. Normal Üniversite Katılımcıları için Merge İşlemi
# Normal katılımcılar; normalized üniversite ismi ile uni_data'daki "Üniversite" üzerinden eşleşecek.
left_key = "Normalized_University"
right_key = "Üniversite"

if left_key not in normal_participants.columns:
    raise KeyError(f"'{left_key}' sütunu bulunamadı. Lütfen katılımcı veri setindeki ilgili sütunu kontrol edin.")
if right_key not in uni_data.columns:
    raise KeyError(f"'{right_key}' sütunu bulunamadı. Lütfen üniversite veri setindeki ilgili sütunu kontrol edin.")

merged_normal = pd.merge(normal_participants, uni_data, left_on=left_key, right_on=right_key, how="left")

# 5. Şehir Bazında Katılımcı Sayısı (Normal Üniversiteler) Hesaplama
# Düzenlenmiş üniversite veri setinde bağlı şehir bilgisi "Şehir" sütununda yer alıyor.
if "Şehir" not in merged_normal.columns:
    raise KeyError("'Şehir' sütunu bulunamadı. Lütfen üniversite veri setindeki sütun adını kontrol edin.")

city_counts = merged_normal.groupby("Şehir").size().reset_index(name="katilimci_sayisi")

# Toplam normal üniversite katılımcı sayısını kontrol edelim:
total_normal = city_counts["katilimci_sayisi"].sum()
print("Normal Üniversite Katılımcı Sayısı (Şehir bazında):", total_normal)

# Ayrıca toplam katılımcı sayısı beklenen 928 ise:
expected_total = 928
actual_total = len(participant_data)
if actual_total != expected_total:
    print(f"UYARI: Toplam katılımcı sayısı {actual_total}, beklenen {expected_total}.")

# 6. GeoJSON Verisi ile Birleştirme (Sadece Normal Üniversiteler için)
# GeoJSON dosyasında şehir isimleri "name" sütununda yer alıyor.
if "Şehir" not in city_coords.columns and "name" in city_coords.columns:
    map_data = city_coords.merge(city_counts, left_on="name", right_on="Şehir", how="left")
elif "Şehir" in city_coords.columns:
    map_data = city_coords.merge(city_counts, on="Şehir", how="left")
else:
    raise KeyError("GeoJSON dosyasında 'Şehir' veya 'name' sütunu bulunamadı.")

# 7. İnteraktif Harita Oluşturma (Normal Üniversiteler)
m = folium.Map(location=[39.0, 35.0], zoom_start=6)

for idx, row in map_data.iterrows():
    geom = row['geometry']
    # Geometri tipi nokta değilse centroid hesapla
    if geom.geom_type != 'Point':
        coord = [geom.centroid.y, geom.centroid.x]
    else:
        coord = [geom.y, geom.x]
    
    katilimci_sayisi = row['katilimci_sayisi'] if pd.notna(row['katilimci_sayisi']) else 0
    # Şehir adı: Eğer "Şehir" yoksa GeoJSON'daki "name" kullanılır.
    city_name = row.get('Şehir', row.get('name', 'Bilinmiyor'))
    
    folium.CircleMarker(
        location=coord,
        radius=5 + katilimci_sayisi,  # Katılımcı sayısına göre marker boyutu ayarlanabilir.
        popup=f"Şehir: {city_name}<br>Katilimci Sayısı: {katilimci_sayisi}",
        color="blue",
        fill=True,
        fill_opacity=0.6
    ).add_to(m)

# Haritayı HTML dosyası olarak kaydedelim.
output_map = "interaktif_harita.html"
m.save(output_map)
print(f"Harita '{output_map}' olarak kaydedildi.")

# Ayrı olarak lise ve yabancı katılımcı sayıları konsola yazdırılıyor.
print("\nKategori Bazında Katılımcı Dağılımı:")
print("Normal Üniversite:", len(normal_participants))
print("Lise:", len(lise_participants))
print("Yabancı:", len(yabanci_participants))
print("Toplam:", len(participant_data))

Katılımcı verisinin sütunları: ['Timestamp', 'Adınız - Soyadınız', 'E-posta adresiniz', 'Telefon numaranız', 'Okulunuz/Kurumunuz', 'Okuduğunuz bölüm', 'Okuyorsanız sınıfınız', 'Etkinliğimizi nereden duydunuz?', 'Konuşmacılarımıza sormak istediğiniz sorularınız ', 'Gizlilik Politikasını ve Şartlar ve Koşulları okudum ve kabul ediyorum.', 'Bu formun talebime yanıt verebilmesi için gönderdiğim bilgileri saklamasına izin veriyorum.', 'Çalışıyorsanız unvanınız', "Etkinliğimizin gelişmelerinden haberdar olmak için WhatsApp grubumuza katılmayı unutmayınız!\nBIOCOM'24 WhatsApp grubu", 'Column 12']
Üniversite verisinin sütunları: ['Şehir', 'Üniversite', 'Tür', 'Geleneksel Kuruluş', 'Resmî Kuruluş']
GeoJSON verisinin sütunları: ['name', 'number', 'geometry']
Normal Üniversite Katılımcı Sayısı: 786
Lise Katılımcı Sayısı: 10
Yabancı Katılımcı Sayısı: 132
Toplam Katılımcı Sayısı: 928
Normal Üniversite Katılımcı Sayısı (Şehir bazında): 490
Harita 'interaktif_harita.html' olarak kaydedildi.

Kategori

In [4]:
m

In [6]:
import pandas as pd

# Yabancı katılımcıları seçelim:
yabanci_participants = participant_data[participant_data["Normalized_University"] == "Yabancı"]

# Yabancı katılımcı sayısını yazdıralım:
print("Yabancı Katılımcı Sayısı:", len(yabanci_participants))

# 1. Yabancı katılımcıların 'Okulunuz/Kurumunuz' sütunundaki değerlerin dağılımını hesaplayalım:
foreign_institutions = yabanci_participants["Okulunuz/Kurumunuz"].value_counts()
print("\nYabancı Katılımcıların Kurum Dağılımı:")
print(foreign_institutions)

# Bu dağılımı, kurum adını ve katılımcı sayısını içeren bir DataFrame'e dönüştürelim:
foreign_institutions_df = foreign_institutions.reset_index()
foreign_institutions_df.columns = ["Kurum", "Katilimci_Sayisi"]

# Kurum dağılımını CSV dosyasına kaydedelim:
foreign_institutions_df.to_csv("yabanci_kurum_listesi.csv", index=False)
print("\nYabancı katılımcıların kurum dağılımı 'yabanci_kurum_listesi.csv' dosyası olarak kaydedildi.")

# 2. Yabancı katılımcıların detaylı listesini oluşturalım:
# İlgili sütunlar: Adınız - Soyadınız, E-posta adresiniz, Okulunuz/Kurumunuz, Okuduğunuz bölüm, Timestamp
columns_of_interest = ["Adınız - Soyadınız", "E-posta adresiniz", "Okulunuz/Kurumunuz", "Okuduğunuz bölüm", "Timestamp"]
yabanci_list_df = yabanci_participants[columns_of_interest]

# Detaylı listeyi CSV dosyasına kaydedelim:
yabanci_list_df.to_csv("yabanci_detay_listesi.csv", index=False)
print("Yabancı katılımcı detay listesi 'yabanci_detay_listesi.csv' dosyası olarak kaydedildi.")

# 3. Ek analiz: Kayıt tarihlerini analiz edelim
# Timestamp sütununu datetime formatına çeviriyoruz.
yabanci_participants["Timestamp"] = pd.to_datetime(yabanci_participants["Timestamp"], errors="coerce")
ilk_tarih = yabanci_participants["Timestamp"].min()
son_tarih = yabanci_participants["Timestamp"].max()
print("\nYabancı Katılımcılar için İlk Kayıt Tarihi:", ilk_tarih)
print("Yabancı Katılımcılar için Son Kayıt Tarihi:", son_tarih)


Yabancı Katılımcı Sayısı: 132

Yabancı Katılımcıların Kurum Dağılımı:
Okulunuz/Kurumunuz
İzmir Yüksek Teknoloji Enstitüsü     12
İzmir Yüksek Teknoloji Enstitüsü      9
İYTE                                  4
Izmir Ekonomi Universitesi            3
ADÜ                                   2
                                     ..
Ytü                                   1
Koçak Farma                           1
BOÜN                                  1
Kafkas ünüversitesi                   1
Uskudar Universitesi                  1
Name: count, Length: 104, dtype: int64

Yabancı katılımcıların kurum dağılımı 'yabanci_kurum_listesi.csv' dosyası olarak kaydedildi.
Yabancı katılımcı detay listesi 'yabanci_detay_listesi.csv' dosyası olarak kaydedildi.

Yabancı Katılımcılar için İlk Kayıt Tarihi: 2024-11-17 18:41:25.194000
Yabancı Katılımcılar için Son Kayıt Tarihi: 2024-12-06 20:15:47.657000


C:\Users\samet\AppData\Local\Temp\ipykernel_20652\2678974389.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  yabanci_participants["Timestamp"] = pd.to_datetime(yabanci_participants["Timestamp"], errors="coerce")
